# Load Text Mining Data

In [ ]:
This notebook reads in the textmining results file ('data/FL_ABS_Chap.xlsx') in Excel format and convert it into a nodes file ('data/FL_ABS_Chap_Nodes.tsv') and an edges file ('data/FL_ABS_Chap_Edges.tsv') in table separated values file format for loading into Neo4j database.

## Import Libraries

In [79]:
#import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# import matplotlib.pyplot as plt 
# import seaborn as sns
# sns.set_style('darkgrid')
# sns.set_palette("colorblind")
# sns.set(rc={'figure.figsize':(11,6)})

import os 
import configparser
import csv
import hashlib

In [80]:
# install or import Neo4j GraphDataScience library
try: 
  from graphdatascience import GraphDataScience
  print('Successfully imported GraphDataScience')
except ModuleNotFoundError:
  !pip3 install graphdatascience
  from graphdatascience import GraphDataScience
  print('installed and imported GraphDataScience')

Successfully imported GraphDataScience


# Custom Functions

In [81]:
# function adapted from Neo4j GDS Fraud Demo Notebook (h/t Zach B.) mainly used for connecting to Neo4j database from Python
def read_neo4j_properties(NEO4J_PROPERTIES_FILE: str=None) -> str:
  '''Parses Neo4j database or Aura connection details from provided .ini filepath.
  Requirements:
    configparser

  Args:
    NEO4J_PROPERTIES_FILE: path to a .ini file
  
  Returns:
    HOST: link to Neo4j or Aura host 
    USERNAME: login username
    PASSWORD: login password 

  Note: The .ini file should use the following syntax
    [NEO4J]
    PASSWORD=<password>
    USERNAME=<database name>
    HOST=<host uri>

  If no path is passed, the function will return the defaults:
    HOST = 'neo4j://localhost'
    USERNAME = 'neo4j'
    PASSWORD = 'password'
  '''

  if NEO4J_PROPERTIES_FILE is not None and os.path.exists(NEO4J_PROPERTIES_FILE):
      config = configparser.RawConfigParser()
      config.read(NEO4J_PROPERTIES_FILE)
      HOST = config['NEO4J']['HOST']
      USERNAME = config['NEO4J']['USERNAME']
      PASSWORD = config['NEO4J']['PASSWORD']
      print('Using HOST, USERNAME, PASSWORD from .ini file')
      return HOST, USERNAME, PASSWORD
  else:
      print('Could not find database properties file, using defaults:')
      HOST = 'neo4j://localhost'
      USERNAME = 'neo4j'
      PASSWORD = 'password'
      print(f'HOST: {HOST} \nUSERHAME: {USERNAME} \nPASSWORD: {PASSWORD}')
      return HOST, USERNAME, PASSWORD 

In [82]:
# function to convert textmining data in TSV format into Nodes and Edges in TSV format
def process_textmining_data_tsv(tm_file_path, nodes_file_path, edges_file_path):
    nodes = []
    edges = []
    with open(tm_file_path, "r", encoding="utf8") as tm_file:
        tsv_reader = csv.reader(tm_file, delimiter="\t")

        # Skip the first row, which is the header
        next(tsv_reader)

        for row in tsv_reader:
            (pmid, pmcid, trigger, arg0_np, arg0_head, arg0_base, arg0_term, arg0_type, arg0_coid, arg1_np, arg1_head, arg1_base, arg1_term, arg1_type, arg1_coid, sent_text, section, relation) = row
            #print(row)
            arg0_id_str = f"{pmid}\t{pmcid}\t{arg0_np}\t{arg0_head}\t{arg0_base}\t{arg0_type}\t{arg0_term}\t{arg0_coid}\t{section}\targ0"
            arg0_id = hashlib.md5(arg0_id_str.encode('utf-8')).hexdigest()
            arg0 = f"{arg0_id}\t{arg0_id_str}"
            
            arg1_id_str = f"{pmid}\t{pmcid}\t{arg1_np}\t{arg1_head}\t{arg1_base}\t{arg1_type}\t{arg1_term}\t{arg1_coid}\t{section}\targ1"
            arg1_id = hashlib.md5(arg1_id_str.encode('utf-8')).hexdigest()
            arg1 = f"{arg1_id}\t{arg1_id_str}"
            
            if arg0 not in nodes:
                nodes.append(arg0)
            
            if arg1 not in nodes:
                nodes.append(arg1)
                
            relation = relation.replace(" ", "_").upper()
            edge = f"{pmid}\t{pmcid}\t{arg0_id}\t{relation}\t{arg1_id}\t{trigger}\t{sent_text}\t{section}"
            
            if edge not in edges:
                edges.append(edge)
                
    tm_file.close()
    
    # open file nodes file for write
    with open(nodes_file_path, "w") as nodes_file:
        # write header
        nodes_header = f"nodeId\tpmid\tpmcid\tnp\thead\tbase\ttype\tterm\tcoid\tsection\targ_type"
        nodes_file.write(nodes_header+"\n")
        # write to file
        for chunk in nodes:
            nodes_file.write(chunk+"\n")
    nodes_file.close()
    
    # open file edges file for write
    with open(edges_file_path, "w") as edges_file:
        # write header
        edges_header = f"pmid\tpmcid\targ0_id\trelation\targ1_id\ttrigger\tsent_text\tsection"
        edges_file.write(edges_header+"\n")
        # write to file
        for chunk in edges:
            edges_file.write(chunk+"\n")
    edges_file.close()

In [83]:
# function to convert textmining data in Excel format into Nodes and Edges in TSV format
def process_textmining_data(tm_excel_file_path, nodes_file_path, edges_file_path):
    nodes = []
    edges = []
    df = pd.read_excel(tm_excel_file_path).fillna('')

    for index, row in df.iterrows():
        pmid = row['PMID/Chapter']
        pmcid = row['PMCID/ISBN']
        trigger = ' '.join(row['trigger'].split())
        arg0_np = ' '.join(row['arg0_np'].split())
        arg0_head = ' '.join(row['arg0_head'].split())
        arg0_base = ' '.join(row['arg0_base'].split())
        arg0_term = ' '.join(row['arg0_matched'].split())
        arg0_type = ' '.join(row['arg0_type'].split())
        arg0_coid = ' '.join(row['arg0_coid'].split())

        arg1_np = ' '.join(row['arg1_np'].split())
        arg1_head = ' '.join(row['arg1_head'].split())
        arg1_base = ' '.join(row['arg1_base'].split())
        arg1_term = ' '.join(row['arg1_matched'].split())
        arg1_type = ' '.join(row['arg1_type'].split())
        arg1_coid = ' '.join(row['arg1_coid'].split())
        
        sent_text = ' '.join(row['sent_text'].split())
        section = ' '.join(row['section'].split())
        relation = ' '.join(row['relation_type'].split())
        
        arg0_id_str = f"{pmid}\t{pmcid}\t{arg0_np}\t{arg0_head}\t{arg0_base}\t{arg0_type}\t{arg0_term}\t{arg0_coid}\t{section}\targ0"
        arg0_id = hashlib.md5(arg0_id_str.encode('utf-8')).hexdigest()
        arg0 = f"{arg0_id}\t{arg0_id_str}"
            
        arg1_id_str = f"{pmid}\t{pmcid}\t{arg1_np}\t{arg1_head}\t{arg1_base}\t{arg1_type}\t{arg1_term}\t{arg1_coid}\t{section}\targ1"
        arg1_id = hashlib.md5(arg1_id_str.encode('utf-8')).hexdigest()
        arg1 = f"{arg1_id}\t{arg1_id_str}"
            
        if arg0 not in nodes:
            nodes.append(arg0)
            
        if arg1 not in nodes:
            nodes.append(arg1)
                
        relation = relation.replace(" ", "_").upper()
        edge = f"{pmid}\t{pmcid}\t{arg0_id}\t{relation}\t{arg1_id}\t{trigger}\t{sent_text}\t{section}"
        
        if edge not in edges:
            edges.append(edge)
    
    # open file nodes file for write
    with open(nodes_file_path, "w") as nodes_file:
        # write header
        nodes_header = f"nodeId\tpmid\tpmcid\tnp\thead\tbase\ttype\tterm\tcoid\tsection\targ_type"
        nodes_file.write(nodes_header+"\n")
        # write to file
        for chunk in nodes:
            nodes_file.write(chunk+"\n")
    nodes_file.close()
    
    # open file edges file for write
    with open(edges_file_path, "w") as edges_file:
        # write header
        edges_header = f"pmid\tpmcid\targ0_id\trelation\targ1_id\ttrigger\tsent_text\tsection"
        edges_file.write(edges_header+"\n")
        # write to file
        for chunk in edges:
            edges_file.write(chunk+"\n")
    edges_file.close()

In [84]:

# Take cleaned textming output file and generate Nodes and Edges files in TSV format for loading into Neo4J
process_textmining_data(tm_excel_file_path='data/FL_ABS_Chap.xlsx', nodes_file_path='data/FL_ABS_Chap_Nodes.tsv', edges_file_path='data/FL_ABS_Chap_Edges.tsv')

# Connect to Neo4j DB
It is recommended to store authentication credentials in a separate file and read them in to the notebook as variables. This code assumes the files are stored in a local auth directory.

In [85]:
# get authentication credentials from local auth file
NEO4J_PROPERTIES_FILE = 'auth/immerse_kg_auth.ini'
HOST, USERNAME, PASSWORD = read_neo4j_properties(NEO4J_PROPERTIES_FILE=NEO4J_PROPERTIES_FILE)

Using HOST, USERNAME, PASSWORD from .ini file


In [ ]:
# connect to neo4j instance 
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=False)

In [87]:
# confirm connection with GDS version 
gds.version()

'2.5.0'

# Clean Textming Contents

In [88]:
# if the step of loading textmining results failed or want to reload textmining results, clearup textmining results from the databas
gds.run_cypher('''
                MATCH (n:TextMining)
                DETACH DELETE n
                ''')

""


# Read textming data into panda dataframe

In [89]:
# read Nodes.tsv
nodes = pd.read_csv('data/FL_ABS_Chap_Nodes.tsv', sep='\t', header=0).fillna('NA') 
nodes.head().to_dict(orient='records')

[{'nodeId': 'ecd04bee9dceaef7fa4938f22aa7554f',
  'pmid': 'chapter_1',
  'pmcid': '978-1-4987-6285-4',
  'np': 'a glycoprotein',
  'head': 'glycoprotein',
  'base': 'a glycoprotein',
  'type': 'product',
  'term': 'glycoprotein',
  'coid': 'COID866322',
  'section': 'CHAPTER 1',
  'arg_type': 'arg0'},
 {'nodeId': '93693ceb75f76e4812c310c1bdba7ccc',
  'pmid': 'chapter_1',
  'pmcid': '978-1-4987-6285-4',
  'np': 'The glycans attached to a glycoprotein',
  'head': 'glycans',
  'base': 'The glycans',
  'type': 'sugar (subset of carbon source)',
  'term': 'glycans',
  'coid': 'COID626759',
  'section': 'CHAPTER 1',
  'arg_type': 'arg1'},
 {'nodeId': 'a5e0d84ea0572fbc2fcb951b74e75094',
  'pmid': 'chapter_1',
  'pmcid': '978-1-4987-6285-4',
  'np': 'the presence of mannose 6-phosphate on the glycan of a number of lysosomal enzymes',
  'head': 'presence',
  'base': 'the presence',
  'type': 'sugar (subset of carbon source)',
  'term': 'glycan',
  'coid': 'COID626759',
  'section': 'CHAPTER 1',

In [90]:
# read Edges.tsv
edges = pd.read_csv('data/FL_ABS_Chap_Edges.tsv', sep='\t', header=0).fillna('NA') 
edges.head().to_dict(orient='records')

[{'pmid': 'chapter_1',
  'pmcid': '978-1-4987-6285-4',
  'arg0_id': 'ecd04bee9dceaef7fa4938f22aa7554f',
  'relation': 'CORRELATED_NOT_SPECIFIED',
  'arg1_id': '93693ceb75f76e4812c310c1bdba7ccc',
  'trigger': 'attached',
  'sent_text': 'The glycans attached to a glycoprotein are structurally heterogeneous.',
  'section': 'CHAPTER 1'},
 {'pmid': 'chapter_1',
  'pmcid': '978-1-4987-6285-4',
  'arg0_id': 'a5e0d84ea0572fbc2fcb951b74e75094',
  'relation': 'CORRELATED_NOT_SPECIFIED',
  'arg1_id': '46d774ee1dfba907a2391c8131f82bab',
  'trigger': 'mediating',
  'sent_text': 'For example, the presence of mannose 6-phosphate on the glycan of a number of lysosomal enzymes is critical in mediating the protein binding to the mannose 6-phosphate receptor on cell plasma membrane for uptake, a step necessary for targeting the enzymes to lysosomes.',
  'section': 'CHAPTER 1'},
 {'pmid': 'chapter_1',
  'pmcid': '978-1-4987-6285-4',
  'arg0_id': 'e0e2a67393582b28c29c16fdf0b34b25',
  'relation': 'CORRELATE

# Load nodes and edges

In [92]:
# Create contraint on TM nodes
gds.run_cypher('''CREATE CONSTRAINT text_mining IF NOT EXISTS FOR (t:TextMining) REQUIRE t.id IS UNIQUE''')

""


In [93]:
# load textming results nodes
gds.run_cypher('''
               UNWIND $node_list AS node
               CALL apoc.merge.node (["TextMining"], {id: node.nodeId, doc_id: node.pmid, doc_id_type: "PMID", pmid: node.pmid, pmcid: node.pmcid, type: node.type, np: node.np, head: node.head, base: node.base, term: node.term, coid: node.coid, section: node.section, arg_type: node.arg_type})
               YIELD node as n 
               RETURN n
              ''', {'node_list': nodes.to_dict('records')})

,n
0,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
1,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
2,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
3,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
4,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
5,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
6,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
7,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
8,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"
9,"(np, coid, section, pmid, type, doc_id, doc_id_type, arg_type, head, term, id, pmcid, base)"


In [94]:
# Add Effector Labels
gds.run_cypher('''
                match(n:TextMining {arg_type:"arg0"}) set n:Effector
                ''')

""


In [95]:
# Add Affected Labels
gds.run_cypher('''
                match(n:TextMining {arg_type:"arg1"}) set n:Affected
                ''')

""


In [96]:
# remove empty term property from TextMining nodes
gds.run_cypher('''
                match(n:TextMining { term: "NA"}) 
                remove n.term
                ''')

""


In [97]:
# remove empty type property from TextMining nodes
gds.run_cypher('''
                match(n:TextMining { type: "NA"}) 
                remove n.type
                ''')

""


In [98]:
# remove unknown type from TM nodes
gds.run_cypher('''
                match(n:TextMining { type: "NYK"}) 
                remove n.type
                ''')

""


In [99]:
# link TextMining nodes to DictionaryConcept by coid
gds.run_cypher('''
                MATCH(t:TextMining), (d:DictionaryConcept)
                WHERE  t.coid = d.id
                MERGE (t)-[:HAS_CANONICAL_NAME]->(d)
                ''')

""


In [100]:
# load TextMining reesults edges
gds.run_cypher('''
               UNWIND $edge_list AS edge
               MATCH (s)
               WHERE s.id = edge.arg0_id
               MATCH (d)
               WHERE d.id = edge.arg1_id
               CALL apoc.merge.relationship(s, edge.relation, {doc_id: edge.pmid, doc_id_type: "PMID", trigger: edge.trigger, sent_text: edge.sent_text, section: edge.section},{}, d,{}) 
               YIELD rel
               RETURN rel
              ''', {'edge_list': edges.to_dict('records')})

,rel
0,"(sent_text, section, trigger, doc_id, doc_id_type)"
1,"(sent_text, section, trigger, doc_id, doc_id_type)"
2,"(sent_text, section, trigger, doc_id, doc_id_type)"
3,"(sent_text, section, trigger, doc_id, doc_id_type)"
4,"(sent_text, section, trigger, doc_id, doc_id_type)"
5,"(sent_text, section, trigger, doc_id, doc_id_type)"
6,"(sent_text, section, trigger, doc_id, doc_id_type)"
7,"(sent_text, section, trigger, doc_id, doc_id_type)"
8,"(sent_text, section, trigger, doc_id, doc_id_type)"
9,"(sent_text, section, trigger, doc_id, doc_id_type)"


In [101]:
# Correct BookChapter nodes
gds.run_cypher('''
MATCH (n:TextMining) 
WHERE n.doc_id STARTS WITH 'chapter'
set n.doc_id='978-1-4987-6285-4', n.doc_id_type="ISBN", n.isbn = '978-1-4987-6285-4' 
remove n.pmid, n.pmcid 
''')

""


In [102]:
# Correct BookChapter edges
gds.run_cypher('''
match()-[r]-() 
WHERE r.doc_id STARTS WITH 'chapter'
set r.doc_id='978-1-4987-6285-4', r.doc_id_type='ISBN'
''')

""


In [103]:
# Create fulltext search index on textmining nodes
gds.run_cypher('''CREATE FULLTEXT INDEX text_mining_search IF NOT EXISTS FOR (n:TextMining) ON EACH [n.base, n.head, n.np, n.doc_id, n.term, n.type]''')

""


In [ ]:
# (optional) exports the whole database incl. indexes as cypher statements to the provided file
gds.run_cypher('''CALL apoc.export.cypher.all('kg_export_after_load_textminig.cypher',{format:'cypher-shell'})''')